In [1]:
# install Pint if necessary
try:
    from pint import UnitRegistry
except ImportError:
    !pip install pint

In [2]:
# download modsim.py if necessary

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ModSimPy/raw/master/' +
         'modsim.py')

In [3]:
# import libraries

from modsim import *
import numpy
import pandas as pd

In [4]:
df = pd.read_csv('sample/suicide-rates-among-young-people.csv')
#df.tail()
#df.head()
#df.info()
df

,Entity,Code,Year,"Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS15-19","Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS20-24","Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS25-34"
0,Afghanistan,AFG,2000,5.773377,8.977578,10.135592
1,Afghanistan,AFG,2001,6.127417,9.883379,10.808245
2,Afghanistan,AFG,2002,5.736194,9.926080,10.616271
3,Afghanistan,AFG,2003,5.589899,9.757331,10.216901
4,Afghanistan,AFG,2004,5.629840,10.310462,10.593060
...,...,...,...,...,...,...
3795,Zimbabwe,ZWE,2015,10.232885,13.056926,17.767336
3796,Zimbabwe,ZWE,2016,9.983424,12.505952,15.922534
3797,Zimbabwe,ZWE,2017,9.773547,12.217892,13.057365
3798,Zimbabwe,ZWE,2018,9.672438,11.963584,12.868393


In [5]:
df_filtered = df[(df['Year'] >= 2000) & (df['Year'] <= 2019)]
df_sorted = df_filtered.sort_values(by=['Entity', 'Year'])

In [6]:
df_grouped = df_filtered.groupby(['Entity', 'Year']).agg({
    'Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS15-19': 'mean',
    'Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS20-24': 'mean',
    'Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS25-34': 'mean'
}).reset_index()

In [7]:
# Tes count by country
changes = {}
for column in df_grouped.columns[2:]:
    changes[column] = df_grouped[column].diff().tolist()
    for entity in df_grouped['Entity'].unique():
        entity_min_year = df_grouped[df_grouped['Entity'] == entity]['Year'].min()
        if entity_min_year == 2000:
            entity_indices = df_grouped.index[df_grouped['Entity'] == entity]
            changes[column][entity_indices[0]] = 0

In [8]:
# Tes count by country
df_changes = pd.DataFrame(changes)
df_changes['Entity'] = df_grouped['Entity']
print(df_changes[0:60])

    Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS15-19  \
0                                            0.000000                                            
1                                            0.354040                                            
2                                           -0.391223                                            
3                                           -0.146295                                            
4                                            0.039941                                            
5                                           -0.225948                                            
6                                           -0.334279                                            
7                                           -0.303834                                            
8                                           -0.372208                                            
9                   

In [9]:
# count all
total_changes = {str(year): 0 for year in range(2000, 2020)}

# Menambahkan nilai perubahan untuk setiap tahun dari seluruh negara
for column in df_grouped.columns[2:]:
    for i, year in enumerate(range(2000, 2020)):
        #print(df_grouped[column][df_grouped['Year']])
        total_changes[str(year)] += sum(df_grouped[column][df_grouped['Year'] == year])

total_changes

{'2000': 6717.58764614,
 '2001': 6526.887709740002,
 '2002': 6422.155027000001,
 '2003': 6268.528138970001,
 '2004': 6215.736750729999,
 '2005': 6218.3023512400005,
 '2006': 6066.676484099999,
 '2007': 6022.519427700001,
 '2008': 5990.335498999999,
 '2009': 5989.247926019999,
 '2010': 5859.586338039998,
 '2011': 5774.415640999998,
 '2012': 5743.841943489999,
 '2013': 5596.780263589999,
 '2014': 5532.081084729998,
 '2015': 5395.127963359999,
 '2016': 5207.601940600001,
 '2017': 5199.35698588,
 '2018': 5034.593952260002,
 '2019': 4957.220710599999}

In [10]:
# Menghitung rata-rata perubahan untuk setiap tahun
mean_changes = {
    year: total_change / len(df_grouped['Entity'].unique()) for year, 
    total_change in total_changes.items()
}

# Menampilkan hasil
print("Rata-rata kenaikan angka kematian untuk seluruh dunia dari tahun 2000 hingga 2019:")
for year, mean_change in mean_changes.items():
    formatted_mean_change = "{:.3f}".format(mean_change)
    print(f"Tahun {year}: {formatted_mean_change}")

Rata-rata kenaikan angka kematian untuk seluruh dunia dari tahun 2000 hingga 2019:
Tahun 2000: 35.356
Tahun 2001: 34.352
Tahun 2002: 33.801
Tahun 2003: 32.992
Tahun 2004: 32.714
Tahun 2005: 32.728
Tahun 2006: 31.930
Tahun 2007: 31.697
Tahun 2008: 31.528
Tahun 2009: 31.522
Tahun 2010: 30.840
Tahun 2011: 30.392
Tahun 2012: 30.231
Tahun 2013: 29.457
Tahun 2014: 29.116
Tahun 2015: 28.395
Tahun 2016: 27.408
Tahun 2017: 27.365
Tahun 2018: 26.498
Tahun 2019: 26.091


In [11]:
df_grouped

,Entity,Year,"Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS15-19","Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS20-24","Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS25-34"
0,Afghanistan,2000,5.773377,8.977578,10.135592
1,Afghanistan,2001,6.127417,9.883379,10.808245
2,Afghanistan,2002,5.736194,9.926080,10.616271
3,Afghanistan,2003,5.589899,9.757331,10.216901
4,Afghanistan,2004,5.629840,10.310462,10.593060
...,...,...,...,...,...
3795,Zimbabwe,2015,10.232885,13.056926,17.767336
3796,Zimbabwe,2016,9.983424,12.505952,15.922534
3797,Zimbabwe,2017,9.773547,12.217892,13.057365
3798,Zimbabwe,2018,9.672438,11.963584,12.868393


In [12]:
mean_changes.keys()

dict_keys(['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019'])

In [13]:
mean_changes.values()

dict_values([35.355724453368424, 34.352040577578954, 33.80081593157895, 32.992253363, 32.71440395121052, 32.72790711178948, 31.929876232105254, 31.69747067210527, 31.528081573684204, 31.522357505368415, 30.839928094947357, 30.39166126842104, 30.230747070999993, 29.456738229421045, 29.116216235421042, 28.395410333473677, 27.408431266315798, 27.365036767789473, 26.49786290663159, 26.09063531894736])